In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,LSTM,Dropout,GRU
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import keras
import random
import numpy as np


In [ ]:
text=open('Sherlock.txt').read().lower()

In [ ]:
print('Given script has '+str(len(text)))

Given script has 581862


In [ ]:
text=text[1302:]
for ch in ['0','1', '2', '3', '4', '5', '6', '7', '8', '9', '!', '"', '$', '%', '&', '~', '`', '(', ')', '*',
          '-', '/', ';', '@', '?', ':', '©', '¢', 'ã', '\xa0', '\n', '\r', '.']:
  if ch in text:
    text=text.replace(ch,' ')

print(set(text))

{'n', "'", 'a', 's', 'é', 'l', 'w', 'p', 'à', 'u', ',', 'x', 'o', 'â', 'h', 'm', 'f', 'z', 'r', 'y', 'c', 'b', 'd', ' ', 'è', 'v', 't', 'e', 'g', 'q', 'j', 'k', 'i'}


In [ ]:
def window_transform(text,window_size,step_size):
  inputs=[]
  outputs=[]

  n_batches=int((len(text)-window_size)/step_size)

  for i in range(n_batches-1):
    a=text[i*step_size:((i*step_size)+window_size)]
    inputs.append(a)
    b=text[(i*step_size)+window_size]
    outputs.append(b)

  return inputs,outputs


In [ ]:
window_size=50
step_size=3
inputs,outputs=window_transform(text,window_size,step_size)

In [ ]:
inputs[500],outputs[500]

('een cocaine and ambition, the drowsiness of the dr', 'u')

In [ ]:
chars=sorted(set(text))

In [ ]:
chars_indices=dict((c,i) for i, c in enumerate(chars))

In [ ]:
indices_chars=dict((i,c) for i,c in enumerate(chars))

In [ ]:
def encode_io_pairs(text,window_size,step_size):
  num_chars=len(chars)

  inputs,outputs=window_transform(text,window_size,step_size)

  X=np.zeros((len(inputs),window_size,num_chars),dtype=np.bool)
  Y=np.zeros((len(inputs),num_chars),dtype=np.bool)

  for i, sentence in enumerate(inputs):
    for t, char in enumerate(sentence):
      X[i,t,chars_indices[char]]=1
    Y[i,chars_indices[outputs[i]]]=1
  
  return X,Y

X,Y=encode_io_pairs(text,window_size,step_size)

In [ ]:
model=Sequential()
model.add(GRU(120,input_shape=(window_size,len(chars))))
model.add(Dropout(0.35))
model.add(Dense(len(chars),activation='relu'))
model.add(Dense(Y.shape[1],activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='RMSprop')

Xsmall=X[:20000,:,:]
Ysmall=Y[:20000,:]

model.fit(Xsmall,Ysmall,batch_size=512,epochs=20)

Epoch 1/20
20000/20000 [==============================] - 19s 972us/step - loss: 3.0889
Epoch 2/20
20000/20000 [==============================] - 19s 965us/step - loss: 2.8265
Epoch 3/20
20000/20000 [==============================] - 19s 965us/step - loss: 2.7051
Epoch 4/20
20000/20000 [==============================] - 19s 964us/step - loss: 2.5904
Epoch 5/20
20000/20000 [==============================] - 19s 964us/step - loss: 2.5048
Epoch 6/20
20000/20000 [==============================] - 19s 965us/step - loss: 2.4380
Epoch 7/20
20000/20000 [==============================] - 19s 957us/step - loss: 2.3895
Epoch 8/20
20000/20000 [==============================] - 19s 957us/step - loss: 2.3470
Epoch 9/20
20000/20000 [==============================] - 19s 954us/step - loss: 2.3140
Epoch 10/20
20000/20000 [==============================] - 19s 955us/step - loss: 2.2821
Epoch 11/20
20000/20000 [==============================] - 19s 954us/step - loss: 2.2539
Epoch 12/20
20000/20000 [=====

In [ ]:
def predict_next_chars(model, input_chars, num_to_predict):     
    pred_chars = ''
    for i in range(num_to_predict):
        # Converting this round's predicted characters to numerical input    
        x_test = np.zeros((1, window_size, len(chars)))
        for t, char in enumerate(input_chars):
            x_test[0, t, chars_indices[char]] = 1.
        # make this round's prediction
        test_predict = model.predict(x_test,verbose = 0)[0]
        # translate numerical prediction back to characters
        r = np.argmax(test_predict)                           
        d = indices_chars[r] 
        # update predicted_chars and input
        pred_chars+=d
        input_chars+=d
        input_chars = input_chars[1:]
    return pred_chars


In [ ]:
# Prediction
start = 89
num_to_predict = 10
input_chars = text[start: start + window_size]
print('Complete sequence:', text[start:start + window_size + num_to_predict])
print('Input sequence:', input_chars)
print('Output sequence:', predict_next_chars(model, input_chars, num_to_predict = num_to_predict))


Complete sequence: otion akin to love for irene adler  all emotions, and that o
Input sequence: otion akin to love for irene adler  all emotions, 
Output sequence: and he mes
